<a href="https://colab.research.google.com/github/sseyonafre/Project/blob/main/AI_14_%EA%B3%A0%EC%84%B8%ED%9D%AC_Section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving user.csv to user.csv


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [8]:
df = pd.read_csv('user.csv',index_col = 1) 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25317 entries, 1 to 25317
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        25317 non-null  int64  
 1   ageClass          25317 non-null  int64  
 2   ageDegree         25317 non-null  int64  
 3   ageGbn            25317 non-null  object 
 4   certGbn           25317 non-null  object 
 5   height            25317 non-null  float64
 6   weight            25317 non-null  float64
 7   crunch            25317 non-null  object 
 8   jump              25317 non-null  object 
 9   trunkFlexion      25317 non-null  object 
 10  IllinoisAgility   25317 non-null  object 
 11  BMI               25317 non-null  float64
 12  situp             25317 non-null  object 
 13  standinglongjump  25317 non-null  object 
 14  standsit          25317 non-null  object 
 15  twominwalk        25317 non-null  object 
 16  threeMwalk        25317 non-null  object

# **EDA**

In [10]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.drop_duplicates(keep='first', inplace=True)
df.replace('-',np.nan,inplace=True)
for i in ['jump','situp','crunch','trunkFlexion','IllinoisAgility','standinglongjump','twominwalk','threeMwalk']:
    df[i]= df[i].astype(float)
df['certGbn'] = df['certGbn'].astype('category')
df.certGbn.replace({'참가증':'4','1등급':'1','2등급':'2','3등급':'3'},inplace=True)

In [11]:
df1 = df.query("ageGbn=='노인'")
df2 = df.query("ageGbn=='성인'")
df3 = df.query("ageGbn=='유소년'")
df4 = df.query("ageGbn=='청소년'")

df1.drop(columns=['crunch','jump','IllinoisAgility','situp','standinglongjump'],inplace=True)
df2.drop(columns=['crunch','jump','IllinoisAgility','standsit','twominwalk','threeMwalk'],inplace=True)
df3.drop(columns=['jump','situp','standsit','twominwalk','threeMwalk'],inplace=True)
df4.drop(columns=['crunch','situp','standsit','twominwalk','threeMwalk'],inplace=True)

df1.standsit = df1.standsit.replace(np.nan,0)
df2.situp = df2.situp.replace(np.nan,0).round(0).astype(int)
df3.crunch = df3.crunch.replace(np.nan,0).round(0).astype(int)
df4.jump = df4.jump.replace(np.nan,0).round(0).astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# **모델링**
> ## 나이구분에 따라 검사항목이 다르기 때문에 모델을 나이구분별로 만들어 준다.


### 1. 노인층 모델링
> 기준모델

In [12]:
#노인층의 모델.

In [13]:
#train, test 셋으로 분리 , target 설정.

target = 'certGbn'

train, test = train_test_split(df1, train_size=0.80, test_size=0.20, 
                              stratify=df1[target], random_state=2)
features1 = df1.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train1 = train[features1]
y_train1 = train[target]
X_test1 = test[features1]
y_test1 = test[target]

In [14]:
#기준모델 
y_train1.value_counts(normalize=True) #불균형. 다중분류.

4    0.539503
3    0.209932
2    0.182844
1    0.067720
Name: certGbn, dtype: float64

In [15]:
#기준모델을 타겟의 최빈값으로 설정.
baseline1 = y_train1.mode()

In [16]:
#기준모델의 정확도를 구해보자.
y_pred = [baseline1] * len(y_test1)
print("기준모델 정확도: ", accuracy_score(y_test1, y_pred))

기준모델 정확도:  0.5405405405405406


> 로지스틱회귀모델

In [17]:
#로지스틱회귀모델을 만들어 교차검증을 수행. #베이지안최적화를 사용해서 최적의 파라미터 찾아보자.

pipe1_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe1_lr, X_train1, y_train1, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.71166156 0.68817721 0.63592587 0.6282492  0.71689419]
교차검증 f1 평균: 0.6761816057638542


> 랜덤포레스트모델

In [18]:
pipe1_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe1_rfc, X_train1, y_train1, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.67980155 0.66558409 0.68522215 0.68680743 0.6945366 ]
교차검증 f1 평균: 0.6823903617498183


> XGBoost모델


In [19]:

pipe1_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe1_xg.fit(X_train1,y_train1)

k = 5

scores = cross_val_score(pipe1_xg, X_train1, y_train1, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.82017403 0.68023217 0.70262244 0.71711813 0.77427354]
교차검증 f1 평균: 0.7388840621642527


### 2. 성인층 모델링
> 기준모델

In [20]:
#train, test 셋으로 분리 , target 설정.

train, test = train_test_split(df2, train_size=0.80, test_size=0.20, 
                              stratify=df2[target], random_state=2)
features2 = df2.drop(columns=['certGbn','exercise','testYm','ageGbn']).columns

X_train2 = train[features2]
y_train2 = train[target]
X_test2 = test[features2]
y_test2 = test[target]

In [21]:
#기준모델 
y_train2.value_counts(normalize=True) #불균형. 다중분류.

4    0.519111
3    0.208268
2    0.186427
1    0.086193
Name: certGbn, dtype: float64

In [22]:
#기준모델을 타겟의 최빈값으로 설정.
baseline2 = y_train2.mode()
y_pred = [baseline2] * len(y_test2)
print("기준모델 정확도: ", accuracy_score(y_test2, y_pred))

기준모델 정확도:  0.5195007800312013


> 로지스틱회귀모델

In [23]:
pipe2_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe2_lr, X_train2, y_train2, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.64076199 0.65860536 0.66421623 0.64354744 0.6676627 ]
교차검증 f1 평균: 0.654958743589874


> 랜덤포레스트모델

In [24]:
pipe2_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe2_rfc, X_train2, y_train2, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74342812 0.77242505 0.70908607 0.70690818 0.73616673]
교차검증 f1 평균: 0.7336028296832364


> XGBoost모델

In [25]:
pipe2_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe2_xg.fit(X_train2,y_train2)

k = 5

scores = cross_val_score(pipe2_xg, X_train2, y_train2, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.73694475 0.74708276 0.69721111 0.70116918 0.75092663]
교차검증 f1 평균: 0.7266668843859089


### 3. 유소년 모델링
> 기준모델

In [26]:
#train, test 셋으로 분리 , target 설정.
train, test = train_test_split(df3, train_size=0.80, test_size=0.20, 
                              stratify=df3[target], random_state=2)
features3 = df3.drop(columns=['ageClass','certGbn','exercise','testYm','ageGbn']).columns

X_train3 = train[features3]
y_train3 = train[target]
X_test3 = test[features3]
y_test3 = test[target]

y_train3.value_counts(normalize=True)

baseline3 = y_train3.mode()
y_pred = [baseline3] * len(y_test3)
print("기준모델 정확도: ", accuracy_score(y_test3, y_pred))

기준모델 정확도:  0.7662337662337663


> 로지스틱회귀모델

In [27]:
pipe3_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe3_lr, X_train3, y_train3, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.72533878 0.72811088 0.75015845 0.71788864 0.7237942 ]
교차검증 f1 평균: 0.7290581899475773


> 랜덤포레스트모델

In [28]:
pipe3_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe3_rfc, X_train3, y_train3, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.82000244 0.8110763  0.82788278 0.80956308 0.80679569]
교차검증 f1 평균: 0.8150640587452884


> XGBoost모델

In [29]:
pipe3_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)

pipe3_xg.fit(X_train3,y_train3)

k = 5

scores = cross_val_score(pipe3_xg, X_train3, y_train3, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.82190689 0.82737683 0.85764958 0.81003079 0.83614339]
교차검증 f1 평균: 0.8306214979168709


###4. 청소년 모델링
> 기준모델

In [30]:

train, test = train_test_split(df4, train_size=0.80, test_size=0.20, 
                              stratify=df4[target], random_state=2)
features4 = df4.drop(columns=['ageClass','certGbn','exercise','testYm','ageGbn']).columns

X_train4 = train[features4]
y_train4 = train[target]
X_test4 = test[features4]
y_test4 = test[target]

y_train4.value_counts(normalize=True)

baseline4 = y_train4.mode()
y_pred = [baseline4] * len(y_test4)
print("기준모델 정확도: ", accuracy_score(y_test4, y_pred))

기준모델 정확도:  0.6786556603773585


> 로지스틱회귀모델

In [31]:
pipe4_lr = make_pipeline(
    SimpleImputer(), 
    StandardScaler(),

    LogisticRegression(n_jobs=-1)
)

k = 5

scores = cross_val_score(pipe4_lr, X_train4, y_train4, cv=k, scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.63378464 0.6623436  0.64848026 0.65891718 0.63402987]
교차검증 f1 평균: 0.6475111117844397


> 랜덤포레스트모델

In [32]:
pipe4_rfc = make_pipeline(
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)
)

pipe4_rfc.fit(X_train4,y_train4)

k = 5

scores = cross_val_score(pipe4_rfc, X_train4, y_train4, cv=k, scoring='f1_weighted') #다중분류에서 f1score 사용하려면 weighted
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.74459204 0.74551987 0.74607986 0.73742879 0.72991863]
교차검증 f1 평균: 0.7407078363622684


> XGBoost모델

In [33]:
pipe4_xg = make_pipeline(
    SimpleImputer(), 
    XGBClassifier(random_state=2, n_jobs=-1)
)


k = 5

scores = cross_val_score(pipe4_xg, X_train4, y_train4, cv=k, 
                         scoring='f1_weighted')
print(f'f1 for {k} folds:', scores)
print(f'교차검증 f1 평균:',scores.mean())

f1 for 5 folds: [0.72774087 0.72277856 0.73227447 0.71479688 0.72103449]
교차검증 f1 평균: 0.723725052480499


### 결과
 >청소년모델을 제외한 나머지는 xgboost로 청소년은 랜덤포레스트로 결정.



# 모델 피클파일로 저장

In [34]:
# 해당모델들을 피클라이브러리를 이용하여 피클파일로 저장.

In [35]:
import pickle 
with open('senior.pickle','wb') as fw:
    pickle.dump(pipe1_xg, fw)

In [36]:
with open('middle.pickle','wb') as fw:
    pickle.dump(pipe2_xg, fw)

In [37]:
with open('underaged.pickle','wb') as fw:
    pickle.dump(pipe3_xg, fw)

In [38]:
with open('youth.pickle','wb') as fw:
    pickle.dump(pipe4_rfc, fw)